In [39]:
import os

import numpy as np
import pandas as pd

import statsmodels.api as sm
from sklearn import linear_model

import seaborn as sns
import matplotlib.pyplot as plt

# Here comes another dataset
This is notebook NO.3, our final notebook in our world happiness series. Here we will introduce our third and final dataset and perform a similar statistical modeling to notebook NO.1

In [40]:
# read in data, do initial cleaning/formatting

geographic_df = pd.read_csv('3 World_countries_data\countries_of_the_world.csv', index_col="Country")
geographic_df = geographic_df[["Region", "Area (sq. mi.)"]]
geographic_df.columns = ['Region', 'Area']
geographic_df.index.name = "Country"

# remove spacing on end of string

geographic_df.Region = [geographic_df.Region[i].replace(" ", "").lower() for i in range(len(geographic_df))]

In [41]:
# This is what our next dataset looks like

geographic_df.head(10)

,Region,Area
Country,,
Afghanistan,asia(ex.neareast),647500
Albania,easterneurope,28748
Algeria,northernafrica,2381740
American Samoa,oceania,199
Andorra,westerneurope,468
Angola,sub-saharanafrica,1246700
Anguilla,latinamer.&carib,102
Antigua & Barbuda,latinamer.&carib,443
Argentina,latinamer.&carib,2766890


# Our first two datasets
If you have followed the previous two notebooks, you know that we worked hard to get some clean country data. We need to access these .csvs

In [42]:
# What my current working directory looks like

os.listdir()

['.ipynb',
 '.ipynb_checkpoints',
 '1 Baseline_happiness_analysis.ipynb',
 '1 World_happiness_report_data',
 '2 Cleaned_data',
 '2 Get_worldbank_data.ipynb',
 '2 Worldbank_data',
 '3 Putting_the_data_together.ipynb',
 '3 World_countries_data']

In [43]:
# What our desired folder contains

os.listdir('2 Cleaned_data')

['worldbank.csv', 'world_happiness.csv']

In [44]:
# Lets access this data from our '2 Cleaned_data' folder

original_df = pd.read_csv('2 Cleaned_data\world_happiness.csv', index_col="Country_Year")
additional_df = pd.read_csv('2 Cleaned_data\worldbank.csv', index_col="Country_Year")

In [45]:
# and if we cleaned them correctly in the last notebook, then there should be no issues when we try to concatenate 

df = pd.concat([original_df, additional_df], axis=1)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  This is separate from the ipykernel package so we can avoid doing imports until


In [46]:
# Previously we added the year to our index of country in order to find matches from data 'a' to data 'b'
# Since we no longer need these type of index, we will do so maintenance and remove it

countries = []
years = []

for index in df.index:
    index_split = index.split("_")
    countries.append(index_split[0])
    years.append(index_split[1])

In [47]:
# apply index changes and save year velues

df.index = countries
df["Year"] = years

# Clean the new data
Naturally, our new data is != our old data. Thus there are many gaps and inconsistencies. Fixing the data is one of the most challenging yet rewarding sections of the data science pipeline. Lets dive in...

In [48]:
# clean up new data's indexes

new_indexes = []
for i in range(len(geographic_df.index)):
    if geographic_df.index[i][-1] == " ":
        new_indexes.append(geographic_df.index[i][:-1])
    else:
        new_indexes.append(geographic_df.index[i])

geographic_df.index = new_indexes

In [49]:
# now we can check what countries exist in our original data but not our new data

for country in np.unique(df.index):
    if country not in geographic_df.index:
        print(country)

Bosnia and Herzegovina
Central African Republic
Congo (Brazzaville)
Congo (Kinshasa)
Ivory Coast
Montenegro
Myanmar
North Macedonia
Palestinian Territories
South Korea
Trinidad and Tobago


In [50]:
# We can do some searching online to fill in the gaps for our missing countries

new_geographic_data = [
    ["Bosnia and Herzegovina", "easterneurope", 19_767],
    ["Central African Republic", "sub-saharanafrica", 240_535],
    ["Congo (Brazzaville)", "sub-saharanafrica", 132_047],
    ["Congo (Kinshasa)", "sub-saharanafrica", 905_400],
    ["Ivory Coast", "sub-saharanafrica", 124_504],
    ["Montenegro", "easterneurope", 5_333],
    ["Myanmar", "asia(ex.neareast)", 261_227],
    ["North Macedonia", "easterneurope", 9_928],
    ["Palestinian Territories", "neareast", 2_402],
    ["South Korea", "asia(ex.neareast)", 38_691],
    ["Trinidad and Tobago", "latinamer.&carib", 1_981]
                    ]

In [51]:
# Lets hold this hand picked data to a new dataframe

new_geographic_df = pd.DataFrame(new_geographic_data, columns=["Country", "Region", "Area"])
new_geographic_df.index = new_geographic_df['Country']
new_geographic_df.drop(['Country'], axis=1, inplace=True)

In [52]:
# we can then append this data to it's master df

geographic_df = geographic_df.append(new_geographic_df)

In [53]:
# Now lets check that no countries from our old df are misssing here

for country in np.unique(df.index):
    if country not in geographic_df.index:
        print(country)

In [54]:
# And if there are countries in our new df that are not represented in our old df, we remove them

for country in geographic_df.index:
    if country not in np.unique(df.index):
        geographic_df.drop([country], axis=0, inplace=True)

In [55]:
# Now we can finally add our regions and area to the original dataframe

for country in geographic_df.index:
    df.loc[country, "Region"] = geographic_df.loc[country, "Region"]
    df.loc[country, "Area"] = geographic_df.loc[country, "Area"]

In [56]:
# Next we will encode our categorical variables

df = pd.get_dummies(df)

In [70]:
# here we can visualize what our new feature space looks like

list(df.columns)

['Happiness',
 'GDP',
 'Family',
 'Health',
 'Freedom',
 'Generosity',
 'Trust_gov',
 'GNI_per_cap',
 'Labor_force',
 'Life_expectancy',
 'Population',
 'Area',
 'Year_2015',
 'Year_2016',
 'Year_2017',
 'Year_2018',
 'Region_asia(ex.neareast)',
 'Region_baltics',
 'Region_c.w.ofind.states',
 'Region_easterneurope',
 'Region_latinamer.&carib',
 'Region_neareast',
 'Region_northernafrica',
 'Region_northernamerica',
 'Region_oceania',
 'Region_sub-saharanafrica',
 'Region_westerneurope']

# Our Final Model
It is what we have all been waiting for: a model! This entire process has led up to this final point and now we get to see the results. It is important to note that we have gained many features since our last model. I wonder if this will help or hinder our predictability?

In [58]:
# Before we model, we must seperate independent and dependent variables

dependent_var = df["Happiness"]
independent_var = df.drop("Happiness", axis=1)

In [59]:
# Next lets create our OLS statsmodel

ind_var_const = sm.add_constant(independent_var)
stats_model_obj = sm.OLS(dependent_var, ind_var_const)
stats_model = stats_model_obj.fit()

C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


In [67]:
# Here we can see our parameters

stats_model.params.round(4)

const                       0.2675
GDP                         0.2363
Family                      0.2844
Health                      0.2632
Freedom                     0.1384
Generosity                  0.0461
Trust_gov                   0.0429
GNI_per_cap                 0.0000
Labor_force                -0.0000
Life_expectancy            -0.0061
Population                  0.0000
Area                        0.0000
Year_2015                   0.0554
Year_2016                   0.0733
Year_2017                   0.0709
Year_2018                   0.0678
Region_asia(ex.neareast)   -0.0363
Region_baltics              0.0036
Region_c.w.ofind.states     0.0044
Region_easterneurope        0.0145
Region_latinamer.&carib     0.1366
Region_neareast             0.0082
Region_northernafrica       0.0570
Region_northernamerica      0.0350
Region_oceania              0.0753
Region_sub-saharanafrica   -0.0759
Region_westerneurope        0.0450
dtype: float64

In [78]:
# Here are the same parameters, this time sorted

stats_model.params.sort_values(ascending=False).round(4)

Family                      0.2844
const                       0.2675
Health                      0.2632
GDP                         0.2363
Freedom                     0.1384
Region_latinamer.&carib     0.1366
Region_oceania              0.0753
Year_2016                   0.0733
Year_2017                   0.0709
Year_2018                   0.0678
Region_northernafrica       0.0570
Year_2015                   0.0554
Generosity                  0.0461
Region_westerneurope        0.0450
Trust_gov                   0.0429
Region_northernamerica      0.0350
Region_easterneurope        0.0145
Region_neareast             0.0082
Region_c.w.ofind.states     0.0044
Region_baltics              0.0036
GNI_per_cap                 0.0000
Area                        0.0000
Population                  0.0000
Labor_force                -0.0000
Life_expectancy            -0.0061
Region_asia(ex.neareast)   -0.0363
Region_sub-saharanafrica   -0.0759
dtype: float64

In [61]:
# some summary statistics

stats_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              Happiness   R-squared:                       0.856
Model:                            OLS   Adj. R-squared:                  0.850
Method:                 Least Squares   F-statistic:                     142.0
Date:                Mon, 10 Aug 2020   Prob (F-statistic):          4.05e-223
Time:                        16:19:04   Log-Likelihood:                 586.92
No. Observations:                 598   AIC:                            -1124.
Df Residuals:                     573   BIC:                            -1014.
Df Model:                          24                                         
Covariance Type:            nonrobust                                         
============================================================================================
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
const                        0.2675      0.119      2.257      0.024       0.035       0.500
GDP                          0.2363      0.044      5.413      0.000       0.151       0.322
Family                       0.2844      0.030      9.406      0.000       0.225       0.344
Health                       0.2632      0.087      3.014      0.003       0.092       0.435
Freedom                      0.1384      0.023      5.915      0.000       0.092       0.184
Generosity                   0.0461      0.026      1.745      0.082      -0.006       0.098
Trust_gov                    0.0429      0.027      1.580      0.115      -0.010       0.096
GNI_per_cap               3.083e-06   5.37e-07      5.742      0.000    2.03e-06    4.14e-06
Labor_force              -3.503e-10   2.44e-10     -1.437      0.151   -8.29e-10    1.28e-10
Life_expectancy             -0.0061      0.003     -2.148      0.032      -0.012      -0.001
Population                1.669e-10   1.14e-10      1.462      0.144   -5.73e-11    3.91e-10
Area                      5.898e-09   2.62e-09      2.253      0.025    7.55e-10     1.1e-08
Year_2015                    0.0554      0.027      2.027      0.043       0.002       0.109
Year_2016                    0.0733      0.031      2.371      0.018       0.013       0.134
Year_2017                    0.0709      0.031      2.274      0.023       0.010       0.132
Year_2018                    0.0678      0.032      2.099      0.036       0.004       0.131
Region_asia(ex.neareast)    -0.0363      0.016     -2.229      0.026      -0.068      -0.004
Region_baltics               0.0036      0.028      0.127      0.899      -0.052       0.059
Region_c.w.ofind.states      0.0044      0.016      0.269      0.788      -0.028       0.036
Region_easterneurope         0.0145      0.018      0.787      0.432      -0.022       0.051
Region_latinamer.&carib      0.1366      0.017      8.193      0.000       0.104       0.169
Region_neareast              0.0082      0.018      0.459      0.647      -0.027       0.044
Region_northernafrica        0.0570      0.025      2.268      0.024       0.008       0.106
Region_northernamerica       0.0350      0.039      0.889      0.374      -0.042       0.112
Region_oceania               0.0753      0.037      2.058      0.040       0.003       0.147
Region_sub-saharanafrica    -0.0759      0.017     -4.495      0.000      -0.109      -0.043
Region_westerneurope         0.0450      0.021      2.176      0.030       0.004       0.086
==============================================================================
Omnibus:                        7.266   Durbin-Watson:                   0.750
Prob(Omnibus):                  0.026   Jarque-Bera (JB):                8.050
Skew:                          -0.181   Prob(JB):       

In [92]:
# Notice that some of our variables here do not pass as having statistical significance,
# Lets access the features with small pvalues

pvals = stats_model.pvalues[1:]
significant_features = pvals[pvals <= 0.05].index

In [93]:
# We can create a new df with the significant features

independent_var = df[significant_features]

In [94]:
# Next lets create our OLS statsmodel

ind_var_const = sm.add_constant(independent_var)
stats_model_obj = sm.OLS(dependent_var, ind_var_const)
stats_model = stats_model_obj.fit()

C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


In [95]:
# Here are the new params in order

stats_model.params.sort_values(ascending=False).round(4)

const                       0.2849
Family                      0.2787
Health                      0.2350
GDP                         0.2118
Freedom                     0.1572
Region_latinamer.&carib     0.1252
Region_oceania              0.0823
Year_2016                   0.0787
Year_2017                   0.0765
Year_2018                   0.0696
Year_2015                   0.0602
Region_northernafrica       0.0477
Region_westerneurope        0.0313
GNI_per_cap                 0.0000
Area                        0.0000
Life_expectancy            -0.0057
Region_asia(ex.neareast)   -0.0399
Region_sub-saharanafrica   -0.0921
dtype: float64

In [96]:
# and our final statistical summary!

stats_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              Happiness   R-squared:                       0.854
Model:                            OLS   Adj. R-squared:                  0.850
Method:                 Least Squares   F-statistic:                     212.3
Date:                Mon, 10 Aug 2020   Prob (F-statistic):          2.57e-230
Time:                        16:33:54   Log-Likelihood:                 582.53
No. Observations:                 598   AIC:                            -1131.
Df Residuals:                     581   BIC:                            -1056.
Df Model:                          16                                         
Covariance Type:            nonrobust                                         
============================================================================================
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
const                        0.2849      0.123      2.313      0.021       0.043       0.527
GDP                          0.2118      0.041      5.201      0.000       0.132       0.292
Family                       0.2787      0.029      9.598      0.000       0.222       0.336
Health                       0.2350      0.084      2.782      0.006       0.069       0.401
Freedom                      0.1572      0.021      7.362      0.000       0.115       0.199
GNI_per_cap               3.788e-06   4.35e-07      8.704      0.000    2.93e-06    4.64e-06
Life_expectancy             -0.0057      0.003     -2.072      0.039      -0.011      -0.000
Area                      5.331e-09   1.87e-09      2.855      0.004    1.66e-09       9e-09
Year_2015                    0.0602      0.029      2.110      0.035       0.004       0.116
Year_2016                    0.0787      0.032      2.463      0.014       0.016       0.141
Year_2017                    0.0765      0.032      2.370      0.018       0.013       0.140
Year_2018                    0.0696      0.033      2.078      0.038       0.004       0.135
Region_asia(ex.neareast)    -0.0399      0.013     -3.019      0.003      -0.066      -0.014
Region_latinamer.&carib      0.1252      0.013      9.417      0.000       0.099       0.151
Region_northernafrica        0.0477      0.023      2.087      0.037       0.003       0.093
Region_oceania               0.0823      0.036      2.290      0.022       0.012       0.153
Region_sub-saharanafrica    -0.0921      0.017     -5.283      0.000      -0.126      -0.058
Region_westerneurope         0.0313      0.017      1.795      0.073      -0.003       0.066
==============================================================================
Omnibus:                        5.032   Durbin-Watson:                   0.734
Prob(Omnibus):                  0.081   Jarque-Bera (JB):                5.264
Skew:                          -0.148   Prob(JB):                       0.0719
Kurtosis:                       3.351   Cond. No.                     1.54e+22
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 1.34e-29. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [ ]:
# And check it out, we have increased our R-squared from the original notebook by finding more data.
# Lets acknowledge that our model is now much more complicated then before, and the addition of
# these independent variables may not have been necessary. Although this may be true, the process of
# finding, cleaning, and merging data is rigorous and treacherous, and an incredible exciting\
# learning experience.

# If you have made it this far, Thank you :)
# - Jamie Voynow